In [ ]:
# Please see the published databricks notebook at: https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/2938235998916852/2021972478966399/5200638329299319/latest.html

In [0]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4137 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1278 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4109 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4578 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [33.5 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 P

In [0]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [0]:
# 1. Read in the AWS S3 bucket into a DataFrame.
home_sales_csv = "/FileStore/tables/home_sales_revised.csv"
home_sales = spark.read.csv(home_sales_csv, inferSchema=True, header=True)


# Show DataFrame
display(home_sales.limit(10))


id,date,date_built,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
f8a53099-ba1c-47d6-9c31-7398aa8f6089,2022-04-08,2016,936923,4,3,3167,11733,2,1,76
7530a2d8-1ae3-4517-9f4a-befe060c4353,2021-06-13,2013,379628,2,2,2235,14384,1,0,23
43de979c-0bf0-4c9f-85ef-96dc27b258d5,2019-04-12,2014,417866,2,2,2127,10575,2,0,0
b672c137-b88c-48bf-9f18-d0a4ac62fb8b,2019-10-16,2016,239895,2,2,1631,11149,2,0,0
e0726d4d-d595-4074-8283-4139a54d0d63,2022-01-08,2017,424418,3,2,2249,13878,2,0,4
5aa00529-0533-46ba-870c-9e881580ef35,2019-01-30,2017,218712,2,3,1965,14375,2,0,7
131492a1-72e2-4a84-bf97-0db14973bfdb,2020-02-08,2017,419199,2,3,2062,8876,2,0,6
8d54a71b-c520-44e5-8ba1-5a84be03ad35,2019-07-21,2010,323956,2,3,1506,11816,1,0,25
e81aacfe-17fe-46b1-a52a-4753d1622b4a,2020-06-16,2016,181925,3,3,2137,11709,2,0,22
2ed8d509-7372-46d5-a9dd-9281a95467d4,2021-08-06,2015,258710,3,3,1918,9666,1,0,25


In [0]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView('home_sales')


In [0]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("SELECT date_built, CONCAT('$', format_number(AVG(price), 2)) as avg_price \
           FROM home_sales \
           WHERE bedrooms=4 \
           GROUP BY date_built \
           ORDER BY date_built").show()


+----------+-----------+
|date_built|  avg_price|
+----------+-----------+
|      2010|$296,800.75|
|      2011|$302,141.90|
|      2012|$298,233.42|
|      2013|$299,999.39|
|      2014|$299,073.89|
|      2015|$307,908.86|
|      2016|$296,050.24|
|      2017|$296,576.69|
+----------+-----------+



In [0]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("SELECT date_built, CONCAT('$', format_number(AVG(price), 2)) as avg_price \
          FROM home_sales \
          WHERE bedrooms=3 AND bathrooms=3 \
          GROUP BY date_built \
          ORDER BY date_built").show()



+----------+-----------+
|date_built|  avg_price|
+----------+-----------+
|      2010|$292,859.62|
|      2011|$291,117.47|
|      2012|$293,683.19|
|      2013|$295,962.27|
|      2014|$290,852.27|
|      2015|$288,770.30|
|      2016|$290,555.07|
|      2017|$292,676.79|
+----------+-----------+



In [0]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("SELECT date_built, CONCAT('$', format_number(AVG(price), 2)) as avg_price \
          FROM home_sales \
          WHERE bedrooms=3 AND bathrooms=3 \
          AND floors=2 AND sqft_living>=2000 \
          GROUP BY date_built \
          ORDER BY date_built").show()


+----------+-----------+
|date_built|  avg_price|
+----------+-----------+
|      2010|$285,010.22|
|      2011|$276,553.81|
|      2012|$307,539.97|
|      2013|$303,676.79|
|      2014|$298,264.72|
|      2015|$297,609.97|
|      2016|$293,965.10|
|      2017|$280,317.58|
+----------+-----------+



In [0]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("SELECT view, CONCAT('$', format_number(AVG(price), 2)) as avg_price \
          FROM home_sales \
          GROUP BY view \
          HAVING AVG(price) >= 350000  \
          ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
+----+-------------+
only showing top 20 rows

--- 1.130544662475586 seconds ---


In [0]:
# 7. Cache the temporary table home_sales.
spark.sql("cache table home_sales")

Out[9]: DataFrame[]

In [0]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

Out[10]: True

In [0]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql("SELECT view, CONCAT('$', format_number(AVG(price), 2)) as avg_price \
          FROM home_sales \
          GROUP BY view \
          HAVING AVG(price) >= 350000  \
          ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
+----+-------------+
only showing top 20 rows

--- 0.584320068359375 seconds ---


In [0]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [0]:
dbutils.fs.ls('home_sales_partitioned')

Out[13]: [FileInfo(path='dbfs:/home_sales_partitioned/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1732678548000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_678959342526232500', name='_committed_678959342526232500', size=35, modificationTime=1732672451000),
 FileInfo(path='dbfs:/home_sales_partitioned/date_built=2010/', name='date_built=2010/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/home_sales_partitioned/date_built=2011/', name='date_built=2011/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/home_sales_partitioned/date_built=2012/', name='date_built=2012/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/home_sales_partitioned/date_built=2013/', name='date_built=2013/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/home_sales_partitioned/date_built=2014/', name='date_built=2014/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/home_sales_partitioned/date_built=2015/', name='date_built=2015/', size=0, modificationTime=0),
 FileInfo(pa

In [0]:
# 11. Read the formatted parquet data.
p_df_p = spark.read.parquet('/home_sales_partitioned')


In [0]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_delays_p')


In [0]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT view, CONCAT('$', format_number(AVG(price), 2)) as avg_price \
          FROM home_sales \
          GROUP BY view \
          HAVING AVG(price) >= 350000  \
          ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
+----+-------------+
only showing top 20 rows

--- 1.0146169662475586 seconds ---


In [0]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

Out[17]: DataFrame[]

In [0]:
# 15. Check if the home_sales is no longer cached
# Verify that the table is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
